# Business Understading

In [127]:
# Import relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [128]:
df = pd.read_csv("final_raw_market_data.csv")
df.head()

,Date,Price (KES/kg),Market Demand Factor,NDVI Index,Market,County,Retail,Temperature (°C),Humidity (%),Rainfall (mm),Crop
0,2024-10-10,30.0,0.874540,0.597815,Kipkaren,Kakamega,50.0,21.714509,72.476894,74.299546,Avocado
1,2024-10-10,75.0,1.450714,0.562021,Kimilili town,Bungoma,100.0,33.369485,44.648370,230.078112,Avocado
2,2024-10-10,56.0,1.231994,0.300125,Kalundu,Kitui,100.0,27.972654,78.401591,87.477061,Avocado
3,2024-10-10,55.0,1.098658,0.724650,Mwatate,Taita-Taveta,60.0,32.789410,81.101858,214.489439,Avocado
4,2024-10-09,25.0,0.656019,0.344342,Sabatia,Kakamega,40.0,24.057928,45.508103,120.197286,Avocado


In [129]:
df.isnull().sum()

Date                      0
Price (KES/kg)          302
Market Demand Factor      0
NDVI Index                0
Market                    0
County                  222
Retail                  800
Temperature (°C)          0
Humidity (%)              0
Rainfall (mm)             0
Crop                      0
dtype: int64

In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51726 entries, 0 to 51725
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Date                  51726 non-null  object 
 1   Price (KES/kg)        51424 non-null  float64
 2   Market Demand Factor  51726 non-null  float64
 3   NDVI Index            51726 non-null  float64
 4   Market                51726 non-null  object 
 5   County                51504 non-null  object 
 6   Retail                50926 non-null  object 
 7   Temperature (°C)      51726 non-null  float64
 8   Humidity (%)          51726 non-null  float64
 9   Rainfall (mm)         51726 non-null  float64
 10  Crop                  51726 non-null  object 
dtypes: float64(6), object(5)
memory usage: 4.3+ MB


In [131]:
df.duplicated().sum()

np.int64(300)

# 2. Data Understanding

In [132]:
from Classes import DataUnderstanding

In [133]:
df = pd.read_csv("final_raw_market_data.csv")
data_report = DataUnderstanding(df)
data_report.full_report()


 First 5 Rows:
         Date  Price (KES/kg)  Market Demand Factor  NDVI Index  \
0  2024-10-10            30.0              0.874540    0.597815   
1  2024-10-10            75.0              1.450714    0.562021   
2  2024-10-10            56.0              1.231994    0.300125   
3  2024-10-10            55.0              1.098658    0.724650   
4  2024-10-09            25.0              0.656019    0.344342   

            Market          County Retail  Temperature (°C)  Humidity (%)  \
0        Kipkaren        Kakamega    50.0         21.714509     72.476894   
1   Kimilili town         Bungoma   100.0         33.369485     44.648370   
2         Kalundu           Kitui   100.0         27.972654     78.401591   
3         Mwatate    Taita-Taveta    60.0         32.789410     81.101858   
4         Sabatia        Kakamega    40.0         24.057928     45.508103   

   Rainfall (mm)       Crop  
0      74.299546   Avocado   
1     230.078112   Avocado   
2      87.477061   Avocado  

# 3. Data Preparation

## 3.1. Data Cleaning

### 3.1.1 Validity

#### 3.1.1.1. Incorrect Data Types: Date & Retail 

In [134]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d', errors='coerce')
df = df[(df['Date'] >= '2021-01-01') & (df['Date'] <= '2024-12-31')]

In [135]:
# Clean the 'Rainfall (mm)' column before conversion
df['Retail'] = df['Retail'].astype(str).str.strip()  # Remove spaces
df['Retail'] = df['Retail'].str.replace(r'[^0-9.]', '', regex=True)  # Keep only numbers and dots
df['Retail'] = pd.to_numeric(df['Retail'], errors='coerce')  # Convert to numeric
# df['Retail']= df['Retail'].fillna(df['Retail'].median()) # fill 

#### 3.1.1.2. Illogical or Invalid Values:

In [138]:
# Temparature - based on the average temperatures in Kenya
df.loc[(df['Temperature (°C)'] < 0) | (df['Temperature (°C)'] > 50), 'Temperature (°C)'] = np.nan
df = df.dropna(subset = 'Temperature (°C)')

In [139]:
# Fixing Humidity (Humidity (%))
df['Humidity (%)'] = df['Humidity (%)'].clip(lower=0, upper=100)


In [140]:
# Fixing Market Demand Factor- This value should not be negative in economics.
df['Market Demand Factor'] = df['Market Demand Factor'].abs()

In [141]:
# Fixing Rainfall (Rainfall (mm))- cannot be negative
df.loc[df['Rainfall (mm)'] < 0, 'Rainfall (mm)'] = 0

In [142]:
# Fixing abnormal values in Retail column based on maerket prices
df = df[(df['Retail'] >= 10) & (df['Retail'] <= 400)]

#### 3.1.1.3. Invalid Dates

In [143]:
from datetime import datetime

today = datetime.today().date()
df.loc[df['Date'] > pd.to_datetime(today), 'Date'] = np.nan # days to today
df['Date'] = pd.to_datetime(df['Date'], errors='coerce', format='%Y-%m-%d')


### 3.1.2 Completeness(Handling Missing Values)

In [147]:
# Price (KES/kg) and County column
df = df.dropna(subset=['Price (KES/kg)', 'County'])
df = df.rename(columns={'Price (KES/kg)': 'Wholesale'})

### 3.1.3. Accuracy(Correcting Outliers & Extreme Values)

In [148]:
# Select numerical columns
num_cols = df.select_dtypes(include=[np.number]).columns
outlier_counts = {}
for col in num_cols:
    Q1 = df[col].quantile(0.25)  
    Q3 = df[col].quantile(0.75) 
    IQR = Q3 - Q1  
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    outlier_counts[col] = len(outliers)
print(outlier_counts)# print the outliers
# Drop outliers since it is a small percentage of the data
df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
df.shape

{'Wholesale': 1518, 'Market Demand Factor': 0, 'NDVI Index': 0, 'Retail': 1844, 'Temperature (°C)': 0, 'Humidity (%)': 0, 'Rainfall (mm)': 0}


(47033, 11)

### 3.1.4. Consistency (Standardizing Categorical Data)

In [150]:
# Checking for duplicates and removing duplicates
df.duplicated().sum()

np.int64(0)

In [151]:
# Checking for any spelling errors in Market and County columns
print(df['County'].unique())

[' Kakamega ' ' Bungoma ' ' Kitui ' ' Taita-Taveta ' ' Embu ' ' Siaya '
 ' Tharaka-Nithi ' ' Kirinyaga ' ' Kilifi ' ' Meru ' ' Muranga ' ' Kwale '
 ' Nyamira ' ' Bomet ' ' Makueni ' ' Trans-Nzoia ' ' Nairobi ' ' Nandi '
 ' Nairobii ' ' Uasin-Gishu ' ' Migori ' ' Kisumu ' ' Nyandarua '
 ' Busia ' ' Kisii ' ' Nakuru ' ' Vihiga ' ' Kericho ' ' Nyeri '
 ' Garissa ' ' Lamu ' ' Kajiado ' ' Laikipia ' ' West-Pokot ' ' Samburu '
 ' Mombasa ' ' Homa-bay ' ' Baringo ' ' Elgeyo-Marakwet ' ' Turkana '
 ' Machakos ' ' Wajir ' ' Mandera ' ' Isiolo ' ' Narok ' ' Tana-River '
 ' Kiambu ' ' Marsabit ']


In [152]:
# Fix known spelling errors
fix_typos = {'Nairobii': 'Nairobi'}
df['County'] = df['County'].replace(fix_typos)


In [153]:
# Check for trailing spaces
df['Market'].unique()

array([' Kipkaren ', ' Kimilili town ', ' Kalundu ', ' Mwatate ',
       ' Sabatia ', ' Embu Town ', ' Makutano Embu ', ' Bondo ',
       ' Chuka ', ' Ngurubani Market ', ' Aram ', ' Gongoni ',
       ' Charo wa Mae ', ' Gakoromone ', ' Kerugoya ',
       ' Kabati - Muranga ', ' Kinango ', ' Kagio ', ' Miruka ',
       ' Mogogosiek Market ', ' Kathonzweni ', ' Bondeni ', ' nairobi ',
       ' Nairobi Wakulima ', ' Khayega ', ' Kibiok ', ' Mumias ',
       ' Makutano Kirinyaga ', ' Mulot ', ' Kakamega Town ', ' Kangeta ',
       ' Eldoret Main ', ' Awendo ', ' Chwele ', ' Kutus ', ' Muhoroni ',
       ' Akala ', ' Mukuyu Market ', ' Wundanyi ', ' Keroka ', ' Mtwapa ',
       ' Mwingi Town ', ' Mwangulu ', ' Kiria-ini Market ',
       ' Taveta Retail Market ', ' Kabiyet Market ',
       ' Kitale Municipality Market ', ' Ndanai Market ',
       ' Butere Livestock Market ', ' Kawangware ', ' Ikonge ',
       ' Bungoma town ', ' Chepterit Market - Nandi ', ' Kangemi Market ',
       ' Kamuk

In [154]:
# Removing any trailing spaces 
df['Market'] = df['Market'].str.strip().str.title().str.replace(r'\s+', ' ', regex=True)

### 3.1.5. Uniformity (Ensuring Data Formatting & Units Are Standardized)

In [155]:
# Convert numerical values to two decimal places
df['Wholesale'] = df['Wholesale'].round(2)
df['Retail'] = df['Retail'].round(2)
df['Market Demand Factor'] = df['Market Demand Factor'].round(2)
# Convert weather  values to one decimal places
df['Temperature (°C)'] = df['Temperature (°C)'].round(1)
df['Humidity (%)'] = df['Humidity (%)'].round(1)
df['Rainfall (mm)'] = df['Rainfall (mm)'].round(1)

In [156]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47033 entries, 0 to 51425
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  47033 non-null  datetime64[ns]
 1   Wholesale             47033 non-null  float64       
 2   Market Demand Factor  47033 non-null  float64       
 3   NDVI Index            47033 non-null  float64       
 4   Market                47033 non-null  object        
 5   County                47033 non-null  object        
 6   Retail                47033 non-null  float64       
 7   Temperature (°C)      47033 non-null  float64       
 8   Humidity (%)          47033 non-null  float64       
 9   Rainfall (mm)         47033 non-null  float64       
 10  Crop                  47033 non-null  object        
dtypes: datetime64[ns](1), float64(7), object(3)
memory usage: 4.3+ MB
